In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [53]:
data = pd.read_csv('../../datas/Store Sales/train.csv')
holidays = pd.read_csv('../../datas/Store Sales/holidays_events.csv')
oil_price = pd.read_csv('../../datas/Store Sales/oil.csv')
transactions = pd.read_csv('../../datas/Store Sales/transactions.csv')


In [54]:
data = data.dropna(ignore_index=True)
holidays = holidays.dropna(ignore_index=True)
oil_price = oil_price.dropna(ignore_index=True)
transactions = transactions.dropna(ignore_index=True)

In [55]:
data['date'] = pd.to_datetime(data['date'])
transactions['date'] = pd.to_datetime(transactions['date'])
oil_price['date'] = pd.to_datetime(oil_price['date'])
holidays['date'] = pd.to_datetime(holidays['date'])

In [56]:
data = data[data['date'].dt.year == 2017]
data = data[data['date'].dt.month.isin([6, 7, 8])]
data = data.rename(columns={'onpromotion': 'on_promotion'})
data

,id,date,store_nbr,family,sales,on_promotion
2865456,2865456,2017-06-01,1,AUTOMOTIVE,6.000,0
2865457,2865457,2017-06-01,1,BABY CARE,0.000,0
2865458,2865458,2017-06-01,1,BEAUTY,1.000,0
2865459,2865459,2017-06-01,1,BEVERAGES,2250.000,9
2865460,2865460,2017-06-01,1,BOOKS,3.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [57]:
oil_price = oil_price[oil_price['date'].dt.year == 2017]
oil_price = oil_price[oil_price['date'].dt.month.isin([6, 7, 8])]
oil_price = oil_price.rename(columns={'dcoilwtico': 'oil_price'})
oil_price

,date,oil_price
1111,2017-06-01,48.32
1112,2017-06-02,47.68
1113,2017-06-05,47.40
1114,2017-06-06,48.13
1115,2017-06-07,45.80
...,...,...
1170,2017-08-25,47.65
1171,2017-08-28,46.40
1172,2017-08-29,46.46
1173,2017-08-30,45.96


In [58]:
holidays

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


EDA

In [59]:
data.groupby(['store_nbr'])['sales'].sum().sort_values(ascending=False)[:5]

store_nbr
44    3.439429e+06
45    3.183741e+06
47    2.954475e+06
3     2.814817e+06
49    2.693547e+06
Name: sales, dtype: float64

EDA on single store

In [60]:
store_44 = data[data['store_nbr'] == 44].drop(columns='store_nbr')
store_44 = store_44.reset_index(drop=True)

In [61]:
store_44['day'] = store_44['date'].dt.day

In [62]:
def transform_value(value):
    if value in [-1, True]:
        return False
    elif value is False:
        return True
    return value

In [63]:
store_44 = pd.merge(store_44, oil_price, on='date', how='left')
store_44['has_oil_price'] = store_44['oil_price'].isna()
store_44['oil_price'] = store_44['oil_price'].fillna(0)
store_44['has_oil_price'] = store_44['has_oil_price'].apply(transform_value)

In [64]:
store_44

,id,date,family,sales,on_promotion,day,oil_price,has_oil_price
0,2866710,2017-06-01,AUTOMOTIVE,12.000,0,1,48.32,True
1,2866711,2017-06-01,BABY CARE,0.000,0,1,48.32,True
2,2866712,2017-06-01,BEAUTY,11.000,0,1,48.32,True
3,2866713,2017-06-01,BEVERAGES,8799.000,19,1,48.32,True
4,2866714,2017-06-01,BOOKS,0.000,0,1,48.32,True
...,...,...,...,...,...,...,...,...
2503,3000388,2017-08-15,POULTRY,909.194,0,15,47.57,True
2504,3000389,2017-08-15,PREPARED FOODS,329.522,0,15,47.57,True
2505,3000390,2017-08-15,PRODUCE,7239.617,10,15,47.57,True
2506,3000391,2017-08-15,SCHOOL AND OFFICE SUPPLIES,345.000,12,15,47.57,True


In [65]:
is_holiday = pd.merge(store_44, holidays, on='date', how='left')
is_holiday = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']]
is_holiday

,type,locale,locale_name,description,transferred
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2602,Holiday,Local,Riobamba,Fundacion de Riobamba,False
2603,Holiday,Local,Riobamba,Fundacion de Riobamba,False
2604,Holiday,Local,Riobamba,Fundacion de Riobamba,False
2605,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [66]:
is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']] = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']].fillna(-1)
is_holiday

,type,locale,locale_name,description,transferred
0,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1
...,...,...,...,...,...
2602,Holiday,Local,Riobamba,Fundacion de Riobamba,False
2603,Holiday,Local,Riobamba,Fundacion de Riobamba,False
2604,Holiday,Local,Riobamba,Fundacion de Riobamba,False
2605,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [67]:
is_holiday['is_holiday'] = is_holiday['transferred'].apply(transform_value)
store_44['is_holiday'] = is_holiday['is_holiday']

In [68]:
store_44

,id,date,family,sales,on_promotion,day,oil_price,has_oil_price,is_holiday
0,2866710,2017-06-01,AUTOMOTIVE,12.000,0,1,48.32,True,False
1,2866711,2017-06-01,BABY CARE,0.000,0,1,48.32,True,False
2,2866712,2017-06-01,BEAUTY,11.000,0,1,48.32,True,False
3,2866713,2017-06-01,BEVERAGES,8799.000,19,1,48.32,True,False
4,2866714,2017-06-01,BOOKS,0.000,0,1,48.32,True,False
...,...,...,...,...,...,...,...,...,...
2503,3000388,2017-08-15,POULTRY,909.194,0,15,47.57,True,False
2504,3000389,2017-08-15,PREPARED FOODS,329.522,0,15,47.57,True,False
2505,3000390,2017-08-15,PRODUCE,7239.617,10,15,47.57,True,False
2506,3000391,2017-08-15,SCHOOL AND OFFICE SUPPLIES,345.000,12,15,47.57,True,False


In [69]:
# Plot using Plotly
fig = px.line(store_44, x='date', y='sales', color='family', title='Sales Over Time for Store 44')

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Sales',
    title='Sales Over Time for Store 44',
    legend_title='Family'
)

# Show the plot
fig.show()

In [70]:
# Plot using Plotly
fig = px.line(store_44, x='date', y='oil_price', title='Oil Price Over Time')

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Oil Price',
    title='Oil Price Over Time'
)

# Show the plot
fig.show()

In [71]:
categories = store_44.value_counts('family')
categories, len(categories)

(family
 AUTOMOTIVE                    76
 HOME APPLIANCES               76
 SCHOOL AND OFFICE SUPPLIES    76
 PRODUCE                       76
 PREPARED FOODS                76
 POULTRY                       76
 PLAYERS AND ELECTRONICS       76
 PET SUPPLIES                  76
 PERSONAL CARE                 76
 MEATS                         76
 MAGAZINES                     76
 LIQUOR,WINE,BEER              76
 LINGERIE                      76
 LAWN AND GARDEN               76
 LADIESWEAR                    76
 HOME CARE                     76
 HOME AND KITCHEN II           76
 BABY CARE                     76
 HOME AND KITCHEN I            76
 HARDWARE                      76
 GROCERY II                    76
 GROCERY I                     76
 FROZEN FOODS                  76
 EGGS                          76
 DELI                          76
 DAIRY                         76
 CLEANING                      76
 CELEBRATION                   76
 BREAD/BAKERY                  76
 BOOKS

Feature Engineering

In [72]:
stores = data.reset_index(drop=True)
stores['year'] = stores['date'].dt.year
stores['month'] = stores['date'].dt.month
stores['day'] = stores['date'].dt.day
stores['day_of_week'] = stores['date'].dt.dayofweek+1
stores['is_weekend'] = stores['day_of_week'] >= 6


stores = pd.merge(stores, oil_price, on='date', how='left')
stores['has_oil_price'] = stores['oil_price'].isna()
stores['oil_price'] = stores['oil_price'].fillna(0)
stores['has_oil_price'] = stores['has_oil_price'].apply(transform_value)

In [73]:
is_holiday = pd.merge(stores, holidays, on='date', how='left')
is_holiday = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']]
is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']] = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']].fillna(-1)
is_holiday['is_holiday'] = is_holiday['transferred'].apply(transform_value)
stores['is_holiday'] = is_holiday['is_holiday']

In [74]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoded_categories = encoder.fit_transform(data[['family']]).toarray()
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(['family']))

stores = stores.join(encoded_df).drop('family', axis=1)

In [75]:
avg_sales = stores.groupby(['date', 'store_nbr'])['sales'].mean()
avg_sales = avg_sales.reset_index()
avg_sales = avg_sales.rename(columns={'sales': 'avg_sales'})
avg_sales

,date,store_nbr,avg_sales
0,2017-06-01,1,342.231848
1,2017-06-01,2,393.080606
2,2017-06-01,3,978.582515
3,2017-06-01,4,321.332576
4,2017-06-01,5,253.877879
...,...,...,...
4099,2017-08-15,50,511.488515
4100,2017-08-15,51,610.744212
4101,2017-08-15,52,563.637758
4102,2017-08-15,53,248.733000


In [76]:
# Plot using Plotly
fig = px.line(avg_sales, x='date', y='avg_sales', color='store_nbr', title='Sales Over Time')

# Update layout
fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Sales',
    title='Sales Over Time for Store 44',
    legend_title='Family'
)

# Show the plot
fig.show()

In [77]:
stores

,id,date,store_nbr,sales,on_promotion,year,month,day,day_of_week,is_weekend,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,2865456,2017-06-01,1,6.000,0,2017,6,1,4,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2865457,2017-06-01,1,0.000,0,2017,6,1,4,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2865458,2017-06-01,1,1.000,0,2017,6,1,4,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2865459,2017-06-01,1,2250.000,9,2017,6,1,4,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2865460,2017-06-01,1,3.000,0,2017,6,1,4,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135427,3000883,2017-08-15,9,438.133,0,2017,8,15,2,False,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
135428,3000884,2017-08-15,9,154.553,1,2017,8,15,2,False,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
135429,3000885,2017-08-15,9,2419.729,148,2017,8,15,2,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
135430,3000886,2017-08-15,9,121.000,8,2017,8,15,2,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [78]:
stores = stores.drop(columns='date')
stores

,id,store_nbr,sales,on_promotion,year,month,day,day_of_week,is_weekend,oil_price,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
0,2865456,1,6.000,0,2017,6,1,4,False,48.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2865457,1,0.000,0,2017,6,1,4,False,48.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2865458,1,1.000,0,2017,6,1,4,False,48.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2865459,1,2250.000,9,2017,6,1,4,False,48.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2865460,1,3.000,0,2017,6,1,4,False,48.32,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135427,3000883,9,438.133,0,2017,8,15,2,False,47.57,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
135428,3000884,9,154.553,1,2017,8,15,2,False,47.57,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
135429,3000885,9,2419.729,148,2017,8,15,2,False,47.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
135430,3000886,9,121.000,8,2017,8,15,2,False,47.57,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [85]:
from sklearn.model_selection import train_test_split

X = stores.drop(columns=['id', 'sales'])
y = stores['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

In [86]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [87]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=21)
rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Mean Squared Error: {rmse}')

Mean Squared Error: 259.72542527955824


In [88]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train_scaled, y_train)

y_pred = xgb_model.predict(X_test_scaled)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Mean Squared Error: {rmse}')

Mean Squared Error: 303.41623589465297


In [89]:
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist_xgb = {
    'n_estimators': randint(50, 200),
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10),
    'min_child_weight': randint(1, 6),
    'subsample': uniform(0.8, 0.2),
    'colsample_bytree': uniform(0.8, 0.2)
}

random_search_xgb = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist_xgb, n_iter=50, cv=3, n_jobs=1, verbose=2, random_state=21)
random_search_xgb.fit(X_train_scaled, y_train)

print(f"Best parameters found: {random_search_xgb.best_params_}")
print(f"Best cross-validation score: {random_search_xgb.best_score_}")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END colsample_bytree=0.8097449761618255, learning_rate=0.06782193195796336, max_depth=3, min_child_weight=1, n_estimators=170, subsample=0.9903197962133926; total time=   0.2s
[CV] END colsample_bytree=0.8097449761618255, learning_rate=0.06782193195796336, max_depth=3, min_child_weight=1, n_estimators=170, subsample=0.9903197962133926; total time=   0.2s
[CV] END colsample_bytree=0.8097449761618255, learning_rate=0.06782193195796336, max_depth=3, min_child_weight=1, n_estimators=170, subsample=0.9903197962133926; total time=   0.2s
[CV] END colsample_bytree=0.9395345971744855, learning_rate=0.04136285439472182, max_depth=3, min_child_weight=5, n_estimators=111, subsample=0.9167182552437199; total time=   0.1s
[CV] END colsample_bytree=0.9395345971744855, learning_rate=0.04136285439472182, max_depth=3, min_child_weight=5, n_estimators=111, subsample=0.9167182552437199; total time=   0.1s
[CV] END colsample_bytree=0.93953

In [90]:
best_xgb = random_search_xgb.best_params_

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=21, **best_xgb)
xgb_model.fit(X_train_scaled, y_train)

y_pred = xgb_model.predict(X_test_scaled)

rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Mean Squared Error: {rmse}')

Mean Squared Error: 209.7299232420501


In [96]:
test_data = pd.read_csv('../../datas/Store Sales/test.csv')
test_data['date'] = pd.to_datetime(test_data['date'])
test_data = test_data.rename(columns={'onpromotion': 'on_promotion'})

In [97]:
stores = test_data.reset_index(drop=True)
stores['year'] = stores['date'].dt.year
stores['month'] = stores['date'].dt.month
stores['day'] = stores['date'].dt.day
stores['day_of_week'] = stores['date'].dt.dayofweek+1
stores['is_weekend'] = stores['day_of_week'] >= 6


stores = pd.merge(stores, oil_price, on='date', how='left')
stores['has_oil_price'] = stores['oil_price'].isna()
stores['oil_price'] = stores['oil_price'].fillna(0)
stores['has_oil_price'] = stores['has_oil_price'].apply(transform_value)

In [98]:
is_holiday = pd.merge(stores, holidays, on='date', how='left')
is_holiday = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']]
is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']] = is_holiday[['type', 'locale', 'locale_name', 'description', 'transferred']].fillna(-1)
is_holiday['is_holiday'] = is_holiday['transferred'].apply(transform_value)
stores['is_holiday'] = is_holiday['is_holiday']

In [99]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoded_categories = encoder.fit_transform(data[['family']]).toarray()
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(['family']))

stores = stores.join(encoded_df).drop('family', axis=1)

In [104]:
test_X = stores.drop(columns=['id', 'date'])
test_X_scaled = scaler.transform(test_X)
y_pred = xgb_model.predict(test_X_scaled)

In [128]:
stores['sales'] = y_pred
stores['sales'] = stores['sales'].clip(lower=0)
stores['sales'] = stores['sales'].astype(float).round(1)
submission = stores[['id', 'sales']]
submission

,id,sales
0,3000888,4.5
1,3000889,0.0
2,3000890,0.0
3,3000891,2097.1
4,3000892,0.6
...,...,...
28507,3029395,380.3
28508,3029396,91.0
28509,3029397,985.2
28510,3029398,125.7


In [129]:
submission.to_csv('sales_submission.csv', index=False)